In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
register_matplotlib_converters()
plt.style.use('seaborn')

In [ ]:
class StaticPlots:
    
    BR_Cases_By_State = None
    BR_Cases_By_City = None
    BR_Cases_Total = None
    states = None
    cities = None
    
    def __init__(self,data):
        
        self.BR_Cases_By_State = data.BR_Cases_By_State
        self.BR_Cases_By_City = data.BR_Cases_By_City
        self.BR_Cases_Total = data.BR_Cases_Total
        self.states = data.states
        
    def TemporalSeries(self, gvalue, gtype='state',hash_value=""):
        
        gvalue = gvalue.decode('utf-8')
        hash_value = hash_value.decode('utf-8')
        
        if gtype == 'state':
            _temp = self.BR_Cases_By_State[self.BR_Cases_By_State[gtype].values == gvalue]
            path = "__temp/__fixed/__tss/tss_"+hash_value+".png"
        else:
            _temp = self.BR_Cases_By_City[self.BR_Cases_By_City[gtype].values == gvalue]
            path = "__temp/__fixed/__tsc/tsc_"+hash_value+".png"
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        Axes.plot_date("date","totalCases",data = _temp,linestyle="solid",label="Total Cases",color='red')
        Axes.plot_date("date","deaths",data=_temp,linestyle="solid", label="Total Deaths",color='purple')

        Axes.set_title("Temporal Series - "+gvalue,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Time (Days)",labelpad=10,fontsize=14)
        Axes.set_xticklabels(_temp["date"].dt.strftime("%d-%m-%Y"))

        Figure.legend(loc="upper left",bbox_to_anchor=(0.1,0.95),fontsize=12)
        Figure.autofmt_xdate()
        Figure.tight_layout()

        Figure.savefig(path)

        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
            
    
    def totalBarState(self,deaths=False,hash_value=""):
        
        hash_value = hash_value.decode('utf-8')
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        
        if deaths == False:
            l_values = _temp.totalCases
            l_index = _temp.state
            label = "Total Cases"
            title = "Number of Infecteds by State"
            path = "__temp/__fixed/tibs_"+hash_value+".png"
        else:
            l_values = _temp.deaths
            l_index = _temp.state
            label = "Total Deaths"
            title = "Number of Deaths by State"
            path = "__temp/__fixed/tdbs_"+hash_value+".png"
            
        df = pd.DataFrame({"state":l_index,"values":l_values})
        df.sort_values("values",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("state","values",data=df,label=label,palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title(title,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("State",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig(path)
        
        del l_values
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df
    
    def totalBarCity(self,state,deaths=False,hash_value=""):
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        _temp = _temp[_temp["state"] == state]
        _temp = _temp[_temp["city"] != "INDEFINIDA/"+state]
        _temp.sort_values("totalCases",ascending=False,inplace=True)
        _temp = _temp.loc[_temp.index[:10]]
        
        if deaths == False:
            l_values = _temp.totalCases
            l_index = _temp.state
            label = "Total Cases"
            title = "Most Infecteds Cities of "+state
            path = "__temp/__fixed/tibc_"+hash_value+".png"
        else:
            l_values = _temp.deaths
            l_index = _temp.state
            label = "Total Deaths"
            title = "Most Death Cases in Cities of "+state
            path = "__temp/__fixed/tdbc_"+hash_value+".png"
            
        l_values = _temp.totalCases
        l_index = _temp.city
            
        df = pd.DataFrame({"city":l_index,"values":l_values})
        df.sort_values("values",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("city","values",data=df,label=label,palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title(title,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("City",labelpad=10,fontsize=14)
        Axes.tick_params('x',rotation=90)
        
        Figure.tight_layout()
        
        Figure.savefig(path)
        
        del l_values
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df
        
    def PieCasesByState(self):
        date = self.BR_Cases_By_State.date.unique()[-1]
        _temp = self.BR_Cases_By_State[data.BR_Cases_By_State['date'] == date].sort_values("totalCases",ascending=False)

        _main = _temp[:5]
        _others = pd.DataFrame([[None,None,"Others",None,None,None,None,sum(_temp[5:]["totalCases"]),None,None]],columns=_main.columns)
        _temp = pd.concat([_main,_others],ignore_index=True)

        colors = ['#FF214B','#FF5745','#FD6865','#FD8978','#FDA978','#DCDCDC']
        
        Figure, Axes = plt.subplots(figsize=(8,8))
        
        Axes.pie(_temp["totalCases"],labels=_temp["state"], shadow=True,radius=2,autopct='%1.1f%%',colors=colors)
        Axes.axis('equal')

        Figure.tight_layout()
        Figure.savefig('__temp/__fixed/pcbs.png')

        del date
        del _temp
        del _main
        del _others
        del colors
        Axes.cla()
        Figure.clear()
        plt.close()
        
#     def MakeStateComparisonGraphs(self):
#         plt.ioff()
#         for i in self.states["State"]:
#             for j in self.states["State"]:
#                 if i != j:
#                     self.ComparisonInfectedBarState(i,j,(i+"_"+j))
#                     self.ComparisonDeathsBarState(i,j,(i+"_"+j))
    
#     def MakeTemporalSeries(self):
#         plt.ioff()
#         for i in self.states["State"]:
#             self.TemporalSeriesState(i,i)
#             for j in self.cities.loc[i]:
#                 if j != False:z
#                     self.TemporalSeriesCity(j,j)
#                 else: 
#                     break

In [11]:
import import_ipynb
import DataLoad

importing Jupyter notebook from DataLoad.ipynb


In [33]:
data = StaticPlots(DataLoad.DataLoad())

In [35]:
data.totalInfectedBarCity("RJ",True)